## Customer Segmentation

### What are we doing?
Create a 2 x 2 value matrix to create 4 customer groups depending on two dimensions, current customer value and potential customer value.

### How are we doing it?
Identify the Highest Value Customer using the RFM model:
- Recency: When was the last time they purchased?
- Frequency: How often and for how long have they purchased?
- Monetary Value/Sales: How much have they purchased?

Only Recency and Monetary will be used for our 2 x 2 matrix.

In [1]:
# Load, filter, clean and aggregate Global Superstore customer data
import matplotlib as plt
import numpy as np
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
df = pd.read_excel('global-superstore.xlsx')
df = df[(df.Segment == 'Consumer') & (df.Country == 'United States')]
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
9,40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,...,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tabl...,4297.644,13,0.4,-1862.3124,865.74,Critical
21,31784,CA-2011-154627,2011-10-29,2011-10-31,First Class,SA-20830,Sue Ann Reed,Consumer,Chicago,Illinois,...,TEC-PH-10001363,Technology,Phones,Apple iPhone 5S,2735.952,6,0.2,341.9940,752.51,High
32,32735,CA-2012-139731,2012-10-15,2012-10-15,Same Day,JE-15745,Joel Eaton,Consumer,Amarillo,Texas,...,FUR-CH-10002024,Furniture,Chairs,HON 5400 Series Task Chairs for Big and Tall,2453.430,5,0.3,-350.4900,690.42,High
34,32543,CA-2011-168494,2011-12-12,2011-12-14,Second Class,NP-18700,Nora Preis,Consumer,Fresno,California,...,FUR-TA-10003473,Furniture,Tables,Bretford Rectangular Conference Table Tops,3610.848,12,0.2,135.4068,683.12,High


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5191 entries, 0 to 51286
Data columns (total 24 columns):
Row ID            5191 non-null int64
Order ID          5191 non-null object
Order Date        5191 non-null datetime64[ns]
Ship Date         5191 non-null datetime64[ns]
Ship Mode         5191 non-null object
Customer ID       5191 non-null object
Customer Name     5191 non-null object
Segment           5191 non-null object
City              5191 non-null object
State             5191 non-null object
Country           5191 non-null object
Postal Code       5191 non-null float64
Market            5191 non-null object
Region            5191 non-null object
Product ID        5191 non-null object
Category          5191 non-null object
Sub-Category      5191 non-null object
Product Name      5191 non-null object
Sales             5191 non-null float64
Quantity          5191 non-null int64
Discount          5191 non-null float64
Profit            5191 non-null float64
Shipping Cost   

In [3]:
# Create RFM Features for each customer
df_RFM = df.groupby('Customer ID').agg({'Order Date': lambda y: (df['Order Date'].max().date() - y.max().date()).days,
                                        'Order ID': lambda y: len(y.unique()),  
                                        'Sales': lambda y: round(y.sum(),2)})
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

,Recency,Frequency,Monetary
Customer ID,,,
RB-19360,96,6,15117.34
AB-10105,41,10,14473.57
KL-16645,47,12,14175.23
SC-20095,349,9,14142.33
HL-15040,43,6,12873.30


### Automate Segmentation

Let's use the 80% quantile to binary customers in terms of Recency, Frequency and Monetary.

- Customers who purchased recently will be binaried as 2, and 1 otherwise.
- Customers who purchase frequently will be binaried as 2, and 1 otherwise.
- Customers who spend a lot will be binaried as 2, and 1 otherwise.

In [4]:
# automate segmentation using 80% quantile for Recency and Monetary
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)

# for Recency, return 2 if in lower 80%, 1 if in top 20%
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)

# for Frequency, return 2 if in top 20%, 1 if in lower 80%
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)

# for Monetary, return 2 if in top 20%, 1 if in lower 80%
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
df_RFM.head()

     Recency  Frequency  Monetary
0.8    222.2        8.0   4070.17


,Recency,Frequency,Monetary,R,F,M
Customer ID,,,,,,
RB-19360,96,6,15117.34,2,1,2
AB-10105,41,10,14473.57,2,2,2
KL-16645,47,12,14175.23,2,2,2
SC-20095,349,9,14142.33,1,2,2
HL-15040,43,6,12873.30,2,1,2


In [5]:
# Calculate RFM (remember only using Recency & Monetary) score and sort customers
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'Customer ID': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM.columns = ['Num Customers','Avg Freq','Avg Recency','Recency Seg','Monetary Seg', 'Avg Monetary']
df_RFM_SUM.head()

,Num Customers,Avg Freq,Avg Recency,Recency Seg,Monetary Seg,Avg Monetary
RMScore,,,,,,
22,73,8,59,2,2,6711.0
21,9,7,425,1,2,8564.0
12,254,6,70,2,1,1902.0
11,73,4,440,1,1,1526.0


In [8]:
## Visualize the value matrix and explore some key numbers

# Average Monetary value for each RM segment
df_RFM_M = df_RFM_SUM.pivot(index='Monetary Seg', columns='Recency Seg', values='Avg Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['Monetary Seg'], ascending = False).set_index(['Monetary Seg'])
df_RFM_M

Recency Seg,1,2
Monetary Seg,,
2,8564.0,6711.0
1,1526.0,1902.0


In [9]:
# Average number of customers for each RM segment
df_RFM_C = df_RFM_SUM.pivot(index='Monetary Seg', columns='Recency Seg', values='Num Customers')
df_RFM_C= df_RFM_C.reset_index().sort_values(['Monetary Seg'], ascending = False).set_index(['Monetary Seg'])
df_RFM_C

Recency Seg,1,2
Monetary Seg,,
2,9,73
1,73,254


In [10]:
# Average Recency for each RM segment
df_RFM_R = df_RFM_SUM.pivot(index='Monetary Seg', columns='Recency Seg', values='Avg Recency')
df_RFM_R= df_RFM_R.reset_index().sort_values(['Monetary Seg'], ascending = False).set_index(['Monetary Seg'])
df_RFM_R

Recency Seg,1,2
Monetary Seg,,
2,425,59
1,440,70


### Breakdown of the customers in each segment

#### Disengaged
- R1 & M2
- Haven't purchased recently, but spent a lot when they did.
- 9 customers in this segment, on average purchased about 425 days ago, and on average worth $8,500.
    - With this low volume of very high value customers, it shouldn't take too much to isolate the disengaging factor and reenage at least some of them.

#### Star
- R2 & M2
- Purchased recently, and purchased a lot.
- 73 customers in this segment, on average purchased about 59 days ago, and on average worth $6,700.
    - Find patterns driving behavior in this group and reinforce what's behind them.

#### Light
- R1 & M1
- Haven't purchased recently, and didn't spend much when they did.
- 73 customers in this segment, on average purchased about 440 days ago, and on average worth $1,500.
    - Minimal retargeting efforts for this group to optimize any low investment opportunities to draw them back in.

#### New
- R2 & M1
- Purchased a little bit (not a lot) recently
- 254 customers in this segment, on average purchased about 70 days ago, and on average worth $1,900.
    - Don't forget to maximize every opportunity to retain these folks
        - What brought them to you in the first place? Did their purchases fulfill their expectations? Find others like them.
